# Librerías

In [2]:
import re 
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
from unidecode import unidecode

# Útiles

In [341]:
def convert_multiple_pages(pages: list) -> str:
    """ 
    Convierte en texto las imagenes de un pdf

    input:
    - pages: lista con imagenes del pdf

    output:
    - texto_completo: str con el texto leído de las imagenes separadas por $$$$$
    """
    texto_completo = ''

    for page in tqdm(pages):
        im_convert = cv2.cvtColor(np.array(page), cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(im_convert, lang = 'spa')

        texto_completo += '\n\n$$$$$$$$$$$\n\n' + text

    return texto_completo

In [342]:
def funcion_limpieza_dict(trans):
    # Limpieza de resultados
    trans_final = {}
    for llave, valor in trans.items():
        llave_nueva = llave.lower().replace("y anama", "yanama")
        llave_nueva = re.sub(r'\s+', ' ', llave_nueva).strip()
        
        
        trans_final[llave_nueva] = valor_nuevo

    return trans_final

# Lectura de datos

In [343]:
# Read image from which text needs to be extracted
pages = convert_from_path('../data/Constitución de 1991_Wayuunaiki.pdf')

In [344]:
pages = np.array(pages, dtype = 'object')

C:\Users\melir\AppData\Local\Temp\ipykernel_22436\91110453.py:1: FutureWarning: The input object of type 'PpmImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PpmImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  pages = np.array(pages, dtype = 'object')


In [345]:
contenido = pages[3:5]

titulos_traducidos = pages[[5, 22, 91, 92, 106]]

articulos_traduccion = pages[23:91]

articulos_traduccion_detallada = pages[93: 107]

palabras_desconocidas_traduccion = pages[107:112]

# Pasar a texto

In [346]:
# Pasar a texto

texto_contenido = convert_multiple_pages(contenido)
texto_titulos_traducidos = convert_multiple_pages(titulos_traducidos)

texto_articulos_traduccion = convert_multiple_pages(articulos_traduccion)

texto_articulos_traduccion_detallada = convert_multiple_pages(articulos_traduccion_detallada)
texto_palabras_desconocidas_traduccion = convert_multiple_pages(palabras_desconocidas_traduccion)

100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


In [347]:
texto = {
    'texto_contenido': texto_contenido,
    'texto_titulos_traducidos': texto_titulos_traducidos,
    'texto_articulos_traduccion': texto_articulos_traduccion,
    'texto_articulos_traduccion_detallada': texto_articulos_traduccion_detallada,
    'texto_palabras_desconocidas_traduccion': texto_palabras_desconocidas_traduccion
}

In [348]:
# Guardar en formato pickle

with open('../data/ocr_constitucion_wayuu/conversion.pickle', 'wb') as f:
    pickle.dump(texto, f)

# Estructura - Texto contenido

In [3]:
with open('/Users/frubana/Desktop/NLP/ProyectoNLP/data/ocr_constitucion_wayuu/conversion.pickle', 'rb') as archivo:
    # Carga los datos desde el archivo pickle
    texto = pickle.load(archivo)

In [4]:
texto

{'texto_contenido': '\n\n$$$$$$$$$$$\n\nPÚTCHI SÚLU\'UKA\nTABLA DE CONTENIDO\n\nSikua\'ipa Súlu\'uwataya E\'iyataa Pútchi\n(Mensajes Introductorios)\n\nETYATIA\nPresentación\n\nSHIKII ALUUWATAWA SA\'U TUÚ MMAKAT\nCarta del Señor Presidente de la República de Colombia\n\nSHIKIPU\'U PERERASION WAYUU\nCarta de la Coordinación Federación Wayuu\n\nSHIKIIPUU "Y ANAMA"\nCarta de la Organización Indígena "Y anama"\n\nNÚUNUIKI CHI SHIKIPU\'UKAI ALATIRAA PUTCCHI SULUUMUIN\nAKARATCHI SUNIUKI WAYUU\nPalabras del Coordinador del Proyecto de la Traducción a siete lenguas indígenas\n\nKonstitusionkat Sipila Tiu Akua\'lIpaka Súlu\'u\nKolonpia\n\n(Traducción de Algunos Apartes de la Constitución Política de\nColombia-1991 al Wayuunaiki)\n\nPÚUTCHI SUPÚLAPUNA\nPREÁMBULO\n\nNAKUA\'IPA WAYUU SA\'U KOLONPIA JEE SAU SUWASHIRÚIN\n\nMMA (De las riquezas y del territorio) :\nArtículos 8,63,72,286,287,289,290,321,329, Transitorios 56 y 38,\nartículos 330,357,361.\n\n23\n\n25\n\n27\n\n\n\n$$$$$$$$$$$\n\nI.\n\nIv

In [349]:
# Preprocesamiento 
texto_contenido =  unidecode(texto_contenido)

In [350]:
trans = {}
before = False 
saltar = False 
text_before = ''
splitted = texto_contenido.split('\n')


for i, line in enumerate(splitted):
    if saltar: 
        saltar = False 
        continue
    if 'nuunuiki'.upper() in line:
        text_before = line + ' ' + splitted[i+1]
        before = True 
        saltar = True
    elif 'Konstitusionkat' in line: 
        break 

    elif not before and line != '':
        text_before = line
        before = True
    elif before and line != '':
        trans[text_before] = line
        before = False
    else: 
        before = False

In [351]:
trans[splitted[i] +  ' ' + splitted[i+1]] = splitted[i+3] +  ' ' + splitted[i+4]  
trans[splitted[i+6]] = splitted[i+7]

In [352]:
texto_final = '\n'.join(splitted[i+8:])

patron = r"([A-Z'\n\s]+?)\s*\(([^)]+)\)"

coincidencias = re.findall(patron, texto_final)

# Crear el diccionario con las coincidencias
diccionario = {llave.strip(): valor.strip() for llave, valor in coincidencias}
diccionario = {llave: valor for llave, valor in diccionario.items() if llave != ''}
diccionario["Karalo'uta Siipula Ayatana"] = 'Documentos para Trabajar'

trans = {**trans, **diccionario}

In [353]:
trans_final = funcion_limpieza_dict(trans)

In [354]:
trans_final

{"putchi sulu'uka": 'tabla de contenido',
 "sikua'ipa sulu'uwataya e'iyataa putchi": 'mensajes introductorios',
 'etyatia': 'presentacion',
 "shikii aluuwatawa sa'u tuu mmakat": 'carta del senor presidente de la republica de colombia',
 "shikipu'u pererasion wayuu": 'carta de la coordinacion federacion wayuu',
 'shikiipuu "yanama"': 'carta de la organizacion indigena "yanama"',
 "nuunuiki chi shikipu'ukai alatiraa putcchi suluumuin akaratchi suniuki wayuu": 'palabras del coordinador del proyecto de la traduccion a siete lenguas indigenas',
 "konstitusionkat sipila tiu akua'lipaka sulu'u kolonpia": 'traduccion de algunos apartes de la constitucion politica de colombia-1991 al wayuunaiki',
 'puutchi supulapuna': 'preambulo',
 "nakua'ipa wayuu sa'u kolonpia jee sau suwashiruin mma": 'de las riquezas y del territorio',
 "sukua'ipa tuu wayuuwaa namuin wayuu jee namui altjuna": 'de los derechos fundamentales de las personas',
 'naa wayuu nama alijuna suunain kootirawaa': 'de los derechos pol

# Titulos traducidos

In [355]:
trans = {"SUKUA'IPA SULU'UWATAYA E'IYATAA PÚTCHI": 'MENSAJES INTRODUCTORIOS',
        "KONSTITUSIONKAT SUÚPÚLA TUÚÚ AKUA'IPAKA SÚLU'U KOLONPIA": "TRADUCCIÓN DE ALGUNOS APARTES DE LA CONSTITUCION POLÍTICA DE COLOMBIA-1991 AL WAYUUNAIK",
        "KARALO'UTA SUPULA AYATANA": "DOCUMENTOS PARA TRABAJAR",
        "SUKUA'IPA AAPAWAA TUÚ ASHAJUUSHIKA SÚLU'U TUÚ KONSTITUSIONKAT SÚPÚLA E'ERAJAA TÚÚ WAYUUNAIKIKA": "PARA ENTENDER MEJOR LO QUE SE HA TRADUCIDO DE LA CONSTITUCIÓN",
        "SUCHIKII TUÚ PÚTCHI NNOJOTKA WE'RAJÚIN" : "LO QUE QUIEREN DECIR LAS PALABRAS DESCONOCIDAS"
        }

In [356]:
trans_final = {**trans_final, **funcion_limpieza_dict(trans)}

In [357]:
trans_final

{"putchi sulu'uka": 'tabla de contenido',
 "sikua'ipa sulu'uwataya e'iyataa putchi": 'mensajes introductorios',
 'etyatia': 'presentacion',
 "shikii aluuwatawa sa'u tuu mmakat": 'carta del senor presidente de la republica de colombia',
 "shikipu'u pererasion wayuu": 'carta de la coordinacion federacion wayuu',
 'shikiipuu "yanama"': 'carta de la organizacion indigena "yanama"',
 "nuunuiki chi shikipu'ukai alatiraa putcchi suluumuin akaratchi suniuki wayuu": 'palabras del coordinador del proyecto de la traduccion a siete lenguas indigenas',
 "konstitusionkat sipila tiu akua'lipaka sulu'u kolonpia": 'traduccion de algunos apartes de la constitucion politica de colombia-1991 al wayuunaiki',
 'puutchi supulapuna': 'preambulo',
 "nakua'ipa wayuu sa'u kolonpia jee sau suwashiruin mma": 'de las riquezas y del territorio',
 "sukua'ipa tuu wayuuwaa namuin wayuu jee namui altjuna": 'de los derechos fundamentales de las personas',
 'naa wayuu nama alijuna suunain kootirawaa': 'de los derechos pol

# Palabras desconocidas

Esta parte como tenía tres columnas no se lograba detectar con el OCR. Por eso, se dividieron los pedazos y se pasaron otra vez por el OCR.

In [358]:
trans = {'administracion': "ajapulu'ujia",
         'arqueologico': "kamma'inru",
         'articulo': "shi'ipajana",
         "autonomia": "laulawaa akua'ipa",
         "autoridad del estado": "sulaulayuu alu'uwatawaa",
         "republica": "sulaulayuu alu'uwatawaa",
         "autoridad indigena": "sulaulayuu wayuu",
         "camara": "kaamara",
         "capitulo": "sa'ata",
         "ciudadano": "wayuu mule'u sulu'u",
         "congreso": "nakotchiraayapala",
         "constitucion": "konstitusionka",
         "cultura": "akua'ipa",
         "deber u obligacion": "tuu a'innuinjatka",
         "democracia": "sukua'ipa",
         "departamento": "sa'ata mma",
         "derecho": "tuu pansawaa",
         "diversidad etnica": "akua'ipa",
         "economia": "sa'inmajia",
         "estado": "sukua'ipa",
         "estado social de derecho":  "sukua'ipa",
         "entidad territorial": "kasa suttiralu",
         "imprescriptible": "wakarolo",
         "inembargable": "nojotsu suikunuinjatuin",
         "ingresos corrientes": "nneru ekeroloka",
         "inalienable": "matujainsalu",
         "juez": "kasalajanakai atuma",
         "jurisdiccion": "sukua'ipa",
         "justicia": "supu'uya",
         "ley": "supulajana",
         "libertad": "tuu taashiika",
         "libertad de conciencia": "taashii a'in",
         "libertad de culto": "taashii akua'ipa",
         "municipio": "munisiipia",
         "nacion": "wayuu",
         "orden justo": "anata supula",
         "participacion": "e'ejirawaa",
         "patrimonio": "sukurolo",
         "plan de desarrollo": "sukua'ipa",
         "poder publico": "sutchuin",
         "politica": "sukua'ipa",
         "provincia": "purowinsia",
         "recursos naturales": "suwashiruin",
         "regimen": "sukua'ipa",
         "representante o senador": "shikiipu'ujana",
         "republica": "anaatushi",
         "resguardo": "su'ummain wayuu",
         "senado": "senaaloka",
         "soberania": "kajuta",
         "sociedad": "naa wayuu",
         "territorio indigena": "su'ummain wayuu",
         "titulo": "sunulia",
         "tutela": "tuu aksion tutela",
         "vocero": "putchimajachi"
         }

# Traducción de artículos

In [378]:
articulos = {
"""A'aka silaulain alu'uwatawa, ni'itain najapiilu'u naa \
ni'ipajanokana silu'umúin tiúii Asanpiilea Nasional \
Konstituyente miinaka wachuntúin niimiin chi \
Maleiwaka'i shiire pa'inwain waya siipula anaa kataa \
o'u, kittirawaa, ayatawaa, wanawaa atujaa, \
taashi wapushua sipula anainjatiin túti wayuukaluirua sa'u \
mmakat latinoamerika munaka niiluwatain tiiti.""" : """Por la autoridad del mando, puesta en manos de sus colaboradores dentro de la \
Asamblea Nacional Constituyente, le pide a Dios para tener nuestros corazones \
unidos, por el bienestar la vida, estar unidos, trabajar, ser iguales, ser libres \
dentro de la mejor manera donde quepamos todos para que las cosas sean buenas y \
Su trabajo ir tras unir de corazón a corazón a todos las personas sobre la tierra \
latinoamericana como le dicen manda esta Constitución Política de Colombia."""
} 

In [379]:
# Sacamos los textos en español. Son textos entre Pedazo y Artículo
patron = r'(Pedazo.*?)\bArtículo|Articulo'

resultados_espaniol = re.findall(patron, texto_articulos_traduccion, 
                                 re.IGNORECASE | re.DOTALL)

In [380]:
len(resultados_espaniol)

56

In [381]:
# Sacamos los textos en wayuu. Son textos entre \n que tienen un (NUM) y terminan en Pedazo
patron = r"(\n(Shi'ipajana|Shi'ipaajana).*?)Pedazo"

resultados_wayuu = re.findall(patron, texto_articulos_traduccion, 
                                 re.IGNORECASE | re.DOTALL)
resultados_wayuu = [r[0] for r in resultados_wayuu]

In [382]:
len(resultados_wayuu)

56

In [383]:
for text_esp, text_wayuu in zip(resultados_espaniol, resultados_wayuu):
    text_wayuu = text_wayuu.lower()
    text_wayuu = re.sub(r'\s+', ' ', text_wayuu).strip()
    text_wayuu = text_wayuu.replace('(', '').replace(')', '').replace('$', '')

    text_esp = text_esp.lower()
    text_esp = re.sub(r'\s+', ' ', text_esp).strip()
    text_esp = text_esp.replace('(', '').replace(')', '').replace('$', '')

    articulos[text_wayuu] = text_esp

# Consolidación

In [384]:
corpus_constitucion = []

for key_w, value_e in articulos.items():
    corpus_constitucion.append({'wayuu': key_w, 'esp': value_e})

for key_e, value_w in trans.items():
    corpus_constitucion.append({'wayuu': value_w, 'esp': key_e})

for key_w, value_e in trans_final.items():
    corpus_constitucion.append({'wayuu': key_w, 'esp': value_e})

In [385]:
corpus_constitucion

[{'wayuu': "A'aka silaulain alu'uwatawa, ni'itain najapiilu'u naa ni'ipajanokana silu'umúin tiúii Asanpiilea Nasional Konstituyente miinaka wachuntúin niimiin chi Maleiwaka'i shiire pa'inwain waya siipula anaa kataa o'u, kittirawaa, ayatawaa, wanawaa atujaa, taashi wapushua sipula anainjatiin túti wayuukaluirua sa'u mmakat latinoamerika munaka niiluwatain tiiti.",
  'esp': 'Por la autoridad del mando, puesta en manos de sus colaboradores dentro de la Asamblea Nacional Constituyente, le pide a Dios para tener nuestros corazones unidos, por el bienestar la vida, estar unidos, trabajar, ser iguales, ser libres dentro de la mejor manera donde quepamos todos para que las cosas sean buenas y Su trabajo ir tras unir de corazón a corazón a todos las personas sobre la tierra latinoamericana como le dicen manda esta Constitución Política de Colombia.'},
 {'wayuu': "shi'ipajana mekiisat 8. naa shikipu'ujanaka alu'uwatawaa miisiiya naa wayuukana napushuaya naimajuinjati siiwashirúin túti mmaka ott

# Traducción detallada

In [339]:
print(texto_articulos_traduccion_detallada)



$$$$$$$$$$$

Shi'ipajana Wanee / Artículo 1 (Simplificado)

//Shi'ipajana/Wanee/(1)/. /Túú/Kolonpia/minaka/shia/

//Pedazo/Uno/(1/. /Esta/Colombia/como-se-dice-la/ella/
/wanee/mma/e'ere/kajutúin/túú/sima/

/una/ tierra / donde/ser-valorados/esta/1a-manera-de-ser/con/
/kamaneejirain/súnain/acheejawaa/túú/kajuta/
/tener-armonía/en/buscar/esto/ser-valorados/
/akua'ipa/siima/painwain/wayaa/súchiirua/tii/ana//.
//a-manera-de-ser/con/estar-unidos/nosotros/detrás/lo/bueno/.//

Pedazo 1. Esta Colombia, como se dice de ella, es una tierra donde se valora la manera de
ser, tener armonía, en busca de estar unidos y de ir detrás de lo bueno.

eo

Shi'ipajana Piama / Artículo 2

//Shi'ipajana/Piama/(2)/. /Sipúla/lotúinjatúin/akua'ipa/
//Pedazo/Dos/(2)/. /Para/ser-recto/la-manera-de-ser/
Núúi/kasakat/acheejanaka/ekira/sipiila/
/esta/cosa-la/se-busca-la/acuerdo/para/
/sa'innajúnuinjatúin/tii/Konstitusionka/shi'ire/palirain/,/
Macerse/esto/Constitución-la/para/confundidos/,/
/kottirain/,/painwain/si